In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user, password
import pymysql
pymysql.install_as_MySQLdb()

### Collect Traffic collision data from France.  CSV to dataframe

In [2]:
csv_characteristics = "FranceCSV/sm_caracteristics.csv"
characteristics_df = pd.read_csv(csv_characteristics, low_memory = False, encoding = 'latin-1')
# drop unnecessary columns and rename the ones we are keeping
characteristics_df.drop(columns=['lum', 'agg', 'int', 'col', 'com', 'adr', 'gps', 'lat', 
                                 'long', 'dep'], inplace=True)
characteristics_df.rename(columns={"Num_Acc": "accident_id", "an": "year", "mois": "month", 
                                   "jour": "day", "atm": "weather"}, inplace=True)
# break the time into hour and minutes to match canada formatting
characteristics_df['hrmn']=characteristics_df['hrmn'].apply(int)
characteristics_df['hrmn']=characteristics_df['hrmn'].apply(str)
characteristics_df['hour'] = characteristics_df['hrmn'].str.slice(start=0, stop=-2)
characteristics_df.drop(columns=['hrmn'], inplace=True)
# clarify the weather coding with real words
characteristics_df['weather'] = characteristics_df['weather'].replace({1: 'Clear', 2: 'Rain', 3: 'Rain',
                                                                      4: 'Snow/Hail', 5: 'Fog/Smoke', 6: 'Strong Wind',
                                                                      7: 'Other', 8: 'Other', 9: 'Other'})
characteristics_df['weather'].fillna(value='Other',inplace=True)
characteristics_df.head()


,Unnamed: 0,accident_id,year,month,day,weather,hour
0,0,2.016000e+11,16.0,2.0,1.0,Other,14
1,1,2.016000e+11,16.0,3.0,16.0,Clear,18
2,2,2.016000e+11,16.0,7.0,13.0,Clear,19
3,3,2.016000e+11,16.0,8.0,15.0,Other,19
4,4,2.016000e+11,16.0,12.0,23.0,Clear,11


In [3]:
characteristics_df.dtypes

Unnamed: 0       int64
accident_id    float64
year           float64
month          float64
day            float64
weather         object
hour            object
dtype: object

In [4]:
csv_places = "FranceCSV/sm_places.csv"
places_df = pd.read_csv(csv_places, low_memory = False, encoding = 'latin-1')
# drop unnecessary columns and rename the ones we are keeping
places_df.drop(columns=['catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'pr', 'pr1','vosp', 'prof', 'plan', 
                        'lartpc', 'larrout', 'infra', 'situ', 'env1'], inplace=True)
places_df.rename(columns={"Num_Acc": "accident_id", "surf": "surface"}, inplace=True)
# clarify the surface coding with real words
places_df['surface'] = places_df['surface'].replace({1: 'Normal', 2: 'Wet', 3: 'Puddles',
                                                                      4: 'Flooded', 5: 'Snow', 6: 'Muddy',
                                                                      7: 'Icy', 8: 'Oil', 9: 'Other'})
places_df['surface'].fillna(value='Other',inplace=True)
places_df.head()

,Unnamed: 0,accident_id,surface
0,0,201600000001,Normal
1,1,201600000002,Normal
2,2,201600000003,Wet
3,3,201600000004,Normal
4,4,201600000005,Normal


In [5]:
places_df.count()

Unnamed: 0     59432
accident_id    59432
surface        59432
dtype: int64

In [6]:
csv_users = "FranceCSV/sm_users.csv"
users_df = pd.read_csv(csv_users, low_memory = False, encoding = 'latin-1')
# drop unnecessary columns and rename the ones we are keeping
users_df.drop(columns=['place', 'catu', 'sexe', 'trajet', 'locp', 'actp', 'etatp', 'an_nais', 'num_veh'], inplace=True)
users_df.rename(columns={"Num_Acc": "accident_id", "secu":"safety_device", 'grav':'fatality'}, inplace=True)
# replace France's coding to 1 for a death and 0 for no death
users_df['fatality'] = users_df['fatality'].replace({1: 0, 2: 1, 3: 0, 4: 0})
# replace France's coding for safety device use, 1 for unknown, 0 for yes, 2 for not used
users_df['safety_device'].fillna(value=0,inplace=True)
users_df['safety_device'] = users_df['safety_device'].replace({1: 1, 2: 1, 3: 1, 10: 1, 20:1, 30:1, 40:1, 90:1, 
                                                 13:1, 23:1, 33:1, 43:1, 93:1})
users_df['safety_device'] = users_df['safety_device'].replace({12: 2, 22: 2, 32: 2, 42: 2, 92:2})
users_df['safety_device'] = users_df['safety_device'].replace({11: 0, 21: 0, 31: 0, 41: 0, 91:0})
# for each accident report on safety usage, 2 = unused device, 1 = unknown usage, 0 = all safety devices available used
user_safety = users_df[['accident_id','safety_device']].groupby('accident_id').max()
# clarify the safety coding with real words
user_safety['safety_device'] = user_safety['safety_device'].replace({0: 'Used', 1: 'Unknown', 2: 'Not Used'})
# for each accident report on total fatalities
user_fatal = users_df[['accident_id','fatality']].groupby('accident_id').sum()
users_group = pd.merge(user_fatal, user_safety, how="left", on='accident_id')
users_group.head()

,fatality,safety_device
accident_id,,
201600000001,0,Used
201600000002,0,Used
201600000003,0,Used
201600000004,0,Unknown
201600000005,0,Used


In [7]:
users_group.count()

fatality         59432
safety_device    59432
dtype: int64

In [8]:
csv_vehicles = "FranceCSV/sm_vehicles.csv"
vehicles_df = pd.read_csv(csv_vehicles, low_memory = False, encoding = 'latin-1')
# drop unnecessary columns and rename the ones we are keeping
vehicles_df.drop(columns=["senc", "occutc", "choc", "obs", "obsm", "manv", "num_veh"], inplace=True)
vehicles_df.rename(columns={'Num_Acc':'accident_id', 'catv':'vehicle_category'}, inplace=True)
# replace France's coding for vehicle type to 1 for light duty vehicle and 0 for any other type
vehicles_df['vehicle_category'] = vehicles_df['vehicle_category'].replace({1: 0, 2: 0, 3: 0, 4: 0, 5:0, 6:0, 7:1, 8:0, 9:0,
                                                                          10:0, 11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:1,
                                                                          18:0, 19:0, 20:0, 21:0, 30:0, 31:0, 32:0, 33:0,
                                                                          34:0, 35:0, 36:0, 37:1, 38:0, 39:0, 40:0, 99:0})
vehicle_light_df = vehicles_df.groupby('accident_id').max()
vehicle_light_df.head()

,Unnamed: 0,vehicle_category
accident_id,,
201600000001,1,1
201600000002,2,1
201600000003,3,1
201600000004,5,1
201600000005,7,1


In [9]:
vehicles_df['vehicle_category'].unique()

array([1, 0], dtype=int64)

In [10]:
c_p_df = pd.merge(characteristics_df, places_df, how="left", on="accident_id")
add_users_df = pd.merge(c_p_df, users_group, how="left", on="accident_id")
france_df = pd.merge(add_users_df, vehicle_light_df, how="left", on=("accident_id"))
all_france_df = france_df[["accident_id", "year", "month", "day", "hour", "weather", 
                           "surface", "fatality", "safety_device", "vehicle_category"]]
all_france_df['year'] += 2000
all_france_df.head()

C:\Users\caeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,accident_id,year,month,day,hour,weather,surface,fatality,safety_device,vehicle_category
0,2.016000e+11,2016.0,2.0,1.0,14,Other,Normal,0,Used,1
1,2.016000e+11,2016.0,3.0,16.0,18,Clear,Normal,0,Used,1
2,2.016000e+11,2016.0,7.0,13.0,19,Clear,Wet,0,Used,1
3,2.016000e+11,2016.0,8.0,15.0,19,Other,Normal,0,Unknown,1
4,2.016000e+11,2016.0,12.0,23.0,11,Clear,Normal,0,Used,1


In [11]:
all_france_df.count()

accident_id         59432
year                59432
month               59432
day                 59432
hour                59432
weather             59432
surface             59432
fatality            59432
safety_device       59432
vehicle_category    59432
dtype: int64

In [12]:
# limit to only accidents involving light duty vehicles (cars)
car_france_df = all_france_df.loc[all_france_df["vehicle_category"] == 1, :]
# limit to only fatal accidents
fatal_france_df = car_france_df.loc[car_france_df["fatality"] > 0, :]
fatal_france_df.drop(columns=["vehicle_category"], inplace=True)
# Add a source column
fatal_france_df['source'] = 'France'
fatal_france_df.head()

C:\Users\caeeb\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\caeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,accident_id,year,month,day,hour,weather,surface,fatality,safety_device,source
13,2.016000e+11,2016.0,4.0,17.0,12,Clear,Normal,1,Used,France
21,2.016000e+11,2016.0,4.0,2.0,10,Clear,Normal,1,Not Used,France
22,2.016000e+11,2016.0,11.0,7.0,15,Rain,Wet,1,Used,France
24,2.016000e+11,2016.0,12.0,4.0,5,Other,Icy,1,Unknown,France
57,2.016000e+11,2016.0,3.0,10.0,4,Clear,Normal,1,Not Used,France


In [13]:
fatal_france_df.count()

accident_id      2581
year             2581
month            2581
day              2581
hour             2581
weather          2581
surface          2581
fatality         2581
safety_device    2581
source           2581
dtype: int64

In [14]:
# largest fatal accident in the dataset
fatal_france_df['fatality'].max()

6

### Export data to SQL database

In [15]:
rds_connection_string = user + ":" + password + "@localhost:3306/traffic_etl"
engine = create_engine(f'mysql://{rds_connection_string}')

In [16]:
# prove that it is working:
engine.table_names()

['accidents']

In [17]:
# No reason to run this again!
fatal_france_df.to_sql(name='accidents', con=engine, if_exists='append', index=False)